# EM算法

EM算法（Expectation-Maximization，期望最大化算法）是一种用于在存在隐变量（或缺失数据）的情形下，求解模型参数最大似然估计（MLE）或最大后验估计（MAP）的迭代优化方法。它广泛应用于混合模型（如混合高斯模型）、隐马尔可夫模型等场景。下面将详细介绍EM算法的基本原理、推导过程以及每个步骤的具体含义。

---

## 1. 问题背景

在很多实际问题中，我们希望对观测数据 $\( X = \{x_1, x_2, \ldots, x_N\} \)$ 建立概率模型，但往往数据生成过程中存在无法直接观测的隐变量 $\( Z = \{z_1, z_2, \ldots, z_N\} \)$（例如混合高斯模型中的类别标签）。假设模型的参数为 $\(\theta\)$，则数据的**完全数据似然函数**为：
$\[
p(X,Z|\theta)
\]$
而由于隐变量 \(Z\) 未观测到，我们只能利用**不完全数据似然函数**：
$\[
p(X|\theta) = \sum_Z p(X,Z|\theta)
\]$
直接最大化 $\(p(X|\theta)\)$ 或其对数似然 $\(\log p(X|\theta)\)$ 往往十分困难，原因在于对数内部存在求和，使得求导和求极值问题变得复杂。

---

## 2. EM算法的基本思想

EM算法的核心思想是利用**交替优化**的思想，将直接求解不完全数据的对数似然问题转化为两个较易处理的子问题：

1. **E步（Expectation Step，期望步骤）**：在当前参数估计 $\(\theta^{(t)}\)$ 下，计算隐变量 $\(Z\)$ 的后验分布 $\(p(Z|X,\theta^{(t)})\)$，并求出完全数据对数似然的期望，即构造一个下界函数 $\(Q\)$：
   $\[
   Q(\theta|\theta^{(t)}) = \mathbb{E}_{Z|X,\theta^{(t)}}\left[\log p(X,Z|\theta)\right]
   \]$
   直观地说，E步利用当前参数对缺失信息进行“填补”，为后续的参数更新提供依据。

2. **M步（Maximization Step，最大化步骤）**：在E步构造的函数 \(Q(\theta|\theta^{(t)})\) 的基础上，对参数 \(\theta\) 进行最大化更新：
   \[
   \theta^{(t+1)} = \arg\max_\theta Q(\theta|\theta^{(t)})
   \]
   通过这一步，我们得到一组使得下界最大的参数更新，从而提高了观测数据的似然值。

这两个步骤不断交替进行，直到参数收敛（例如对数似然的变化小于某个阈值）。

---

## 3. EM算法的数学推导

### 3.1 利用Jensen不等式构造下界

考虑对数似然：
\[
\log p(X|\theta) = \log \sum_Z p(X,Z|\theta)
\]
引入任意的概率分布 \(q(Z)\)（满足 \(\sum_Z q(Z)=1\) 且 \(q(Z) > 0\)），有：
\[
\log p(X|\theta) = \log \sum_Z q(Z) \frac{p(X,Z|\theta)}{q(Z)}
\]
利用Jensen不等式（对于凹函数，\(\log \mathbb{E}[Y] \geq \mathbb{E}[\log Y]\)），可以得到：
\[
\log p(X|\theta) \geq \sum_Z q(Z) \log \frac{p(X,Z|\theta)}{q(Z)}
\]
记这个下界为：
\[
\mathcal{L}(q,\theta) = \sum_Z q(Z) \log p(X,Z|\theta) - \sum_Z q(Z) \log q(Z)
\]
可以证明，当我们取 \(q(Z) = p(Z|X,\theta^{(t)})\) 时，这个下界对 \(\theta\) 最为紧密，此时：
\[
Q(\theta|\theta^{(t)}) = \mathbb{E}_{Z|X,\theta^{(t)}}\left[\log p(X,Z|\theta)\right]
\]
而第二项（熵项）与 \(\theta\) 无关，因此在M步只需最大化 \(Q\) 函数。

### 3.2 E步的详细内容

在第 \(t\) 次迭代中，给定当前参数 \(\theta^{(t)}\)，计算每个可能的隐变量配置 \(Z\) 的后验概率：
\[
q(Z) = p(Z|X,\theta^{(t)})
\]
然后计算期望：
\[
Q(\theta|\theta^{(t)}) = \sum_Z p(Z|X,\theta^{(t)}) \log p(X,Z|\theta)
\]
对于连续型隐变量，求和可以替换为积分。这一步实际上是在“软填补”缺失数据，为参数更新提供充分信息。

### 3.3 M步的详细内容

在M步中，固定 \(Q(\theta|\theta^{(t)})\) 的表达形式，对参数 \(\theta\) 进行最大化：
\[
\theta^{(t+1)} = \arg\max_\theta Q(\theta|\theta^{(t)})
\]
这一步往往比直接最大化 \(\log p(X|\theta)\) 要简单，因为完全数据的对数似然 \(\log p(X,Z|\theta)\) 通常具有较简单的形式（例如对数似然分解成各部分参数独立的和）。

---

## 4. EM算法的性质

- **单调性**：EM算法保证每次迭代后，观测数据对数似然 \(\log p(X|\theta)\) 都不会减小。这是由于：
  \[
  \log p(X|\theta^{(t+1)}) \geq \mathcal{L}(q,\theta^{(t+1)}) \geq \mathcal{L}(q,\theta^{(t)}) = \log p(X|\theta^{(t)})
  \]
  因此，EM算法的迭代过程总是沿着似然值上升的方向前进，直至收敛到局部极大值（或鞍点）。

- **局部最优性**：EM算法只能保证收敛到局部最优解或鞍点，因此参数的初始化对结果有较大影响。

- **适用范围**：只要能够计算出 \(p(Z|X,\theta^{(t)})\) 和 \(Q(\theta|\theta^{(t)})\) 的解析形式，EM算法就可以应用；在很多模型中，如混合模型和隐马尔可夫模型中，都能方便地应用EM算法。

---

## 5. 以混合高斯模型为例

在混合高斯模型中：
- **隐变量 \(Z\)**：表示每个数据点 \(x_i\) 属于哪一个高斯成分，其取值 \(z_i \in \{1,2,\dots,K\}\)。
- **完全数据似然**：
  \[
  p(x_i, z_i=k|\theta) = \pi_k\, \mathcal{N}(x_i|\mu_k, \Sigma_k)
  \]
  其中 \(\theta = \{\pi_k, \mu_k, \Sigma_k\}_{k=1}^K\)。

**E步**：
计算每个数据点属于每个成分的后验概率（责任）：
\[
\gamma(z_{ik}) = P(z_i=k|x_i,\theta^{(t)}) = \frac{\pi_k^{(t)}\,\mathcal{N}(x_i|\mu_k^{(t)}, \Sigma_k^{(t)})}{\sum_{j=1}^{K} \pi_j^{(t)}\,\mathcal{N}(x_i|\mu_j^{(t)}, \Sigma_j^{(t)})}
\]
这相当于利用当前参数对隐变量 \(z_i\) 进行“软分配”。

**M步**：
利用E步得到的责任 \(\gamma(z_{ik})\) 更新参数：
- 更新混合系数：
  \[
  \pi_k^{(t+1)} = \frac{1}{N}\sum_{i=1}^N \gamma(z_{ik})
  \]
- 更新均值：
  \[
  \mu_k^{(t+1)} = \frac{\sum_{i=1}^N \gamma(z_{ik})\, x_i}{\sum_{i=1}^N \gamma(z_{ik})}
  \]
- 更新协方差矩阵：
  \[
  \Sigma_k^{(t+1)} = \frac{\sum_{i=1}^N \gamma(z_{ik})\, (x_i-\mu_k^{(t+1)})(x_i-\mu_k^{(t+1)})^T}{\sum_{i=1}^N \gamma(z_{ik})}
  \]

---

## 6. 总结

EM算法通过以下两个交替步骤实现参数估计：
1. **E步**：利用当前参数估计，计算隐变量的后验分布，为每个数据点“填补”缺失信息。
2. **M步**：在填补了隐变量的基础上，最大化完全数据对数似然的期望，从而更新参数。

这种方法使得即使在存在缺失数据或隐变量的情况下，也能通过分步优化的方式近似求解最大似然问题。尽管EM算法可能收敛到局部最优解，并且依赖于初始参数，但其简单高效的结构使其在统计学习和机器学习中得到了广泛应用。

In [1]:
# 导入numpy库 
import numpy as np

### EM算法过程函数定义
def em(data, thetas, max_iter=30, eps=1e-3):
    '''
    输入：
    data：观测数据
    thetas：初始化的估计参数值
    max_iter：最大迭代次数
    eps：收敛阈值
    输出：
    thetas：估计参数
    '''
    # 初始化似然函数值
    ll_old = -np.infty
    for i in range(max_iter):
        ### E步：求隐变量分布
        # 对数似然
        log_like = np.array([np.sum(data * np.log(theta), axis=1) for theta in thetas])
        # 似然
        like = np.exp(log_like)
        # 求隐变量分布
        ws = like/like.sum(0)
        # 概率加权
        vs = np.array([w[:, None] * data for w in ws])
        ### M步：更新参数值
        thetas = np.array([v.sum(0)/v.sum() for v in vs])
        # 更新似然函数
        ll_new = np.sum([w*l for w, l in zip(ws, log_like)])
        print("Iteration: %d" % (i+1))
        print("theta_B = %.2f, theta_C = %.2f, ll = %.2f" 
              % (thetas[0,0], thetas[1,0], ll_new))
        # 满足迭代条件即退出迭代
        if np.abs(ll_new - ll_old) < eps:
            break
        ll_old = ll_new
    return thetas

In [2]:
# 观测数据，5次独立试验，每次试验10次抛掷的正反次数
# 比如第一次试验为5次正面5次反面
observed_data = np.array([(5,5), (9,1), (8,2), (4,6), (7,3)])
# 初始化参数值，即硬币B的正面概率为0.6，硬币C的正面概率为0.5
thetas = np.array([[0.6, 0.4], [0.5, 0.5]])
thetas = em(observed_data, thetas, max_iter=30, eps=1e-3)

Iteration: 1
theta_B = 0.71, theta_C = 0.58, ll = -32.69
Iteration: 2
theta_B = 0.75, theta_C = 0.57, ll = -31.26
Iteration: 3
theta_B = 0.77, theta_C = 0.55, ll = -30.76
Iteration: 4
theta_B = 0.78, theta_C = 0.53, ll = -30.33
Iteration: 5
theta_B = 0.79, theta_C = 0.53, ll = -30.07
Iteration: 6
theta_B = 0.79, theta_C = 0.52, ll = -29.95
Iteration: 7
theta_B = 0.80, theta_C = 0.52, ll = -29.90
Iteration: 8
theta_B = 0.80, theta_C = 0.52, ll = -29.88
Iteration: 9
theta_B = 0.80, theta_C = 0.52, ll = -29.87
Iteration: 10
theta_B = 0.80, theta_C = 0.52, ll = -29.87
Iteration: 11
theta_B = 0.80, theta_C = 0.52, ll = -29.87
Iteration: 12
theta_B = 0.80, theta_C = 0.52, ll = -29.87


In [3]:
thetas

array([[0.7967829 , 0.2032171 ],
       [0.51959543, 0.48040457]])